In [14]:
import numpy as np
import struct
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from torchvision import datasets, transforms
import time

In [15]:
mnist_training, mnist_testing = tf.keras.datasets.mnist.load_data()
training_data = np.float16(mnist_training[0])  # Returns np.array
training_labels = np.asarray(mnist_training[1], dtype=np.int32)
testing_data = np.float16(mnist_testing[0])  # Returns np.array
testing_labels = np.asarray(mnist_testing[1], dtype=np.int32)

In [16]:
feature_columns = [tf.feature_column.numeric_column("images", shape=[28, 28])]

train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"images": training_data},
    y= training_labels,
    num_epochs = None,
    batch_size = 50,
    shuffle = True
)

test_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x = {"images": testing_data},
    y = testing_labels,
    num_epochs = 1,
    shuffle = False
)

In [17]:
def repeatedTrainingAndTesting(model_name, dropout, batch_normalization, M):
    
          
    classifier = tf.estimator.DNNClassifier(
        feature_columns = feature_columns,
        hidden_units = [100, 100],
        optimizer = 'Adam',
        n_classes = 10,
        dropout = dropout,
        model_dir="./models/"+model_name,
        batch_norm = batch_normalization
    )
    
    accuracies = []
    totalTime = 0
    
    for i in range(M):
        
        print('_______________________________________________________________________ M='+str(i+1))
        
        # TRAIN
        start_time = time.time()
        classifier.train(input_fn = train_input_fn, steps=5000)
        end_time = time.time()
        duration = end_time - start_time
        print("Training time: --- %s seconds ---" % (duration))
        totalTime += duration
        
        # TEST
        start_time = time.time()
        accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
        end_time = time.time()
        duration = end_time - start_time
        print("Testing Accuracy: {0:f}%\n".format(accuracy_score*100))
        print("Testing time: --- %s seconds ---" % (duration))
        accuracies.append(accuracy_score)
        totalTime += duration
        
    return (accuracies, totalTime)

# MLP

In [18]:
m = 50

## With Dropout, With Batch-Normalization

In [ ]:
(accuracies_DO_BN, time_DO_BN) = repeatedTrainingAndTesting(model_name = "mlp_DO_BN", dropout = 0.2, batch_normalization = True, M=m)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/mlp_DO_BN', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
_______________________________________________________________________ M=1
INF

# Without Dropout, With Batch-Normalization

In [ ]:
(accuracies_BN, time_BN) = repeatedTrainingAndTesting(model_name = "mlp_BN", dropout = None, batch_normalization = True, M=m)

# With Dropout, Without Batch-Normalization

In [ ]:
(accuracies_DO, time_DO) = repeatedTrainingAndTesting(model_name = "mlp_DO", dropout = 0.2, batch_normalization = False, M=m)

# Without Dropout, Without Batch-Normalization

In [ ]:
(accuracies_none, time_none) = repeatedTrainingAndTesting(model_name = "mlp_none", dropout = None, batch_normalization = False, M=m)

# Save Results

In [ ]:
data = {'Accuracy_DO_BN': accuracies_DO_BN, 'Time_DO_BN': time_DO_BN } , 
        'Accuracy_BN' : accuracies_BN, 'Time_BN' : time_BN, 
        'Accuracy_DO' : accuracies_DO, 'Time_DO': time_DO, 
        'Accuracy_NONE': accuracies_none, 'Time_NONE': time_none}

df = pd.DataFrame(data)

df.to_csv('outputs/MLP.csv')